**pgvector = Postgres feature (extension).**

**PGVectorStore = LangChain’s vector database wrapper on top of pgvector.**

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [2]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="nomic-embed-text")

In [3]:
from langchain_postgres import PGEngine, PGVectorStore
PG_HOST = os.environ.get("PGHOST")
PG_USER = os.environ.get("PGUSER")
PG_PASSWORD = os.environ.get("PGPASSWORD")
PG_DATABASE = os.environ.get("PGDATABASE")
PG_PORT = os.environ.get("PGPORT", "5432")

# Use async driver (not psycopg2!)
connection_url = f"postgresql+asyncpg://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

# Create engine
pg_engine = PGEngine.from_connection_string(url=connection_url)


In [6]:
# Define table name and vector size
TABLE_NAME = "vectorstores"
VECTOR_SIZE = 768 # This depends on your embedding model

# Create the table asynchronously
await pg_engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=VECTOR_SIZE,
)
# metadata_columns=[Column("len", "INTEGER")], For custom column name

In [7]:
vector_store = PGVectorStore.create_sync(
    engine=pg_engine,
    table_name="vectorstores",
    embedding_service=embedding,
)

# Add the Text in the PG Vector

In [8]:
# Add some texts
vector_store.add_texts([
    "LangChain makes AI apps easier to build.",
    "PGVector enables semantic search inside Postgres."
])


['4372e251-1729-4174-b63a-b0a2779b8d68',
 'b0718cbd-435b-47db-be5c-10091ae87663']

In [9]:
# Perform a similarity search
results = vector_store.similarity_search("is postgres support semantic search?", k=1)
for r in results:
    print(r.page_content)

PGVector enables semantic search inside Postgres.


# Add documents  
Add documents to the vector store. Metadata is stored in a JSON column, see "Create a custom Vector Store" to store metadata to be used for filters.

In [10]:
import uuid

from langchain_core.documents import Document

docs = [
    Document(
        id=str(uuid.uuid4()),
        page_content="Red Apple",
        metadata={"description": "red", "content": "1", "category": "fruit"},
    ),
    Document(
        id=str(uuid.uuid4()),
        page_content="Banana Cavendish",
        metadata={"description": "yellow", "content": "2", "category": "fruit"},
    ),
    Document(
        id=str(uuid.uuid4()),
        page_content="Orange Navel",
        metadata={"description": "orange", "content": "3", "category": "fruit"},
    ),
]

await vector_store.aadd_documents(docs)

['cb564269-14ff-423a-b4cf-f9768db18f3f',
 '74db8ec0-2177-42ad-8add-081c4f932423',
 '0a5b6fee-84a1-4810-bf3d-290f1b866c3f']

# Delete the Document

In [12]:
ids = [str(uuid.uuid4()) for _ in docs]
await vector_store.adelete([ids[1]])

True

# Search for documents

In [14]:
query = "Which is red fruit ?"
docs = await vector_store.asimilarity_search(query)
print(docs)

[Document(id='cb564269-14ff-423a-b4cf-f9768db18f3f', metadata={'description': 'red', 'content': '1', 'category': 'fruit'}, page_content='Red Apple'), Document(id='74db8ec0-2177-42ad-8add-081c4f932423', metadata={'description': 'yellow', 'content': '2', 'category': 'fruit'}, page_content='Banana Cavendish'), Document(id='0a5b6fee-84a1-4810-bf3d-290f1b866c3f', metadata={'description': 'orange', 'content': '3', 'category': 'fruit'}, page_content='Orange Navel'), Document(id='4372e251-1729-4174-b63a-b0a2779b8d68', metadata={}, page_content='LangChain makes AI apps easier to build.')]


# Search for documents by vector
Search for similar documents using a vector embedding.

In [15]:
query_vector = embedding.embed_query(query)
docs = await vector_store.asimilarity_search_by_vector(query_vector, k=2)
print(docs)

[Document(id='cb564269-14ff-423a-b4cf-f9768db18f3f', metadata={'description': 'red', 'content': '1', 'category': 'fruit'}, page_content='Red Apple'), Document(id='74db8ec0-2177-42ad-8add-081c4f932423', metadata={'description': 'yellow', 'content': '2', 'category': 'fruit'}, page_content='Banana Cavendish')]


# Add a Index
Speed up vector search queries by applying a vector index.

In [16]:
from langchain_postgres.v2.indexes import HNSWIndex, IVFFlatIndex

index = IVFFlatIndex(name="Demo")
await vector_store.aapply_vector_index(index)

# index = HNSWIndex(name="my-hnsw-index")
# await store.aapply_vector_index(index)

In [17]:
await vector_store.areindex("Demo")  # Re-index using the default index name

In [ ]:
#await vector_store.adrop_vector_index("Demo")  # Delete index using the index name

In [ ]:
query = "I'd like a fruit."
docs = await vector_store.asimilarity_search(query)
for i in docs:
   print(i.page_content)

Red Apple
Banana Cavendish
Orange Navel
LangChain makes AI apps easier to build.


# Create a Vector Store using existing table

In [ ]:
# Set an existing table name
TABLE_NAME = "products"
# SCHEMA_NAME = "my_schema"

# Initialize PGVectorStore
custom_store = await PGVectorStore.create(
    engine=pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=embedding,
    # Connect to existing VectorStore by customizing below column names
    id_column="product_id",
    content_column="description",
    embedding_column="embed",
    metadata_columns=["name", "category", "price_usd", "quantity", "sku", "image_url"],
    metadata_json_column="metadata",
)

"""Optional: If the embed column is newly created or has different dimensions than supported by embedding model, it is required to one-time add the embeddings for the old records, like this:

ALTER TABLE products ADD COLUMN embed vector(768) DEFAULT NULL"""

# Hybrid Search with PGVectorStore  

**Hybrid search combines semantic search (embeddings) and keyword search (TSV) to deliver more accurate results.**

**Semantic search: understands meaning beyond exact words, good for synonyms and context, but may miss exact keyword matches.**

**Keyword search: finds exact matches quickly, but lacks understanding of meaning or variations.**

**👉 Together, hybrid search balances contextual understanding and precise keyword matching for more relevant results.**

In [32]:
from langchain_postgres import PGVectorStore
from langchain_postgres.v2.hybrid_search_config import (
    HybridSearchConfig,
    reciprocal_rank_fusion,
)

hybrid_search_config = HybridSearchConfig(
    tsv_column="hybrid_description",
    tsv_lang="pg_catalog.english",
    fusion_function=reciprocal_rank_fusion,
    fusion_function_parameters={
        "rrf_k": 60,
        "fetch_top_k": 10,
    },
)
TABLE_NAME = "hybrid_searching_product"

await pg_engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    vector_size=768,
    id_column="product_id",
    content_column="description",
    embedding_column="embed",
    metadata_columns="name",
    metadata_json_column="metadata",
    hybrid_search_config=hybrid_search_config,
    store_metadata=True,
)


In [ ]:

vs_hybrid = await PGVectorStore.create(
    pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=embedding,
    # Connect to existing VectorStore by customizing below column names
    id_column="product_id",
    content_column="description",
    embedding_column="embed",
    metadata_columns="name",
    metadata_json_column="metadata",
    hybrid_search_config=hybrid_search_config,
)



In [ ]:
# Fetch documents from the previously created store to fetch product documents
docs = await custom_store.asimilarity_search("products", k=5)
# Add data normally to the hybrid search vector store, which will also add the tsv values in tsv_column
await vs_hybrid.aadd_documents(docs)

# Use hybrid search
hybrid_docs = await vs_hybrid.asimilarity_search("products", k=5)
print(hybrid_docs)